In [ ]:
#| default_exp record_validation

In [ ]:
#|exporti 

import pandas as pd
from pandas import DataFrame as PandasDataFrame
from typing import *
from archetypon.base_model import BaseModel,GenericModel,DataFrame
from pydantic import parse_obj_as,ValidationError,validator
from pydantic.utils import update_not_none

In [ ]:
#|exporti 

def parse_dataframe_rows_as(
    model:Type[BaseModel],
    df:PandasDataFrame
)->PandasDataFrame:
    """Uses .parse_obj() method of Pydantic's `BaseModel` to validate rows of a dataframe."""
    
    # convert dataframe to a series of dictionaries
    # drop NaN values because pydantic doesn't consider them to be 'None', and that interferes with validation
    series_of_dicts = df.apply(
        lambda row: row.dropna().to_dict(),
        axis=1
    )
    
    #convert the series of dicts to a series of parsed models
    errors = []
    def parse_row(row):
        try:
            validated = parse_obj_as(model,row)
            return validated
        except ValidationError as e:
            raw_errors = e.raw_errors
            
            errors.extend(raw_errors)
            
            return BaseModel()
        return validated
    
    series_of_models = series_of_dicts.apply(
        lambda x: parse_row(x),
    )
    if len(errors)>0:
        raise ValidationError(errors=errors,model=model)
    # from model back to dictionaries and then back to Series
    validated = series_of_models.apply(
        lambda x: x.dict(),
    ).apply(pd.Series)
    
    return validated

In [ ]:
import datetime as dt

In [ ]:
class Model(BaseModel):
    number: int
    string: str 
    date: dt.date
    
dataframe = pd.DataFrame({
    'number':['1',2,3e0],
    'string':['a','b',3],
    'date':['1994-06-11',dt.date.today(),dt.datetime.today()],
},dtype='object')
display(dataframe)
dataframe.dtypes

,number,string,date
0,1,a,1994-06-11
1,2,b,2023-02-10
2,3.0,3,2023-02-10 14:03:05.866280


number    object
string    object
date      object
dtype: object

In [ ]:
validated = parse_dataframe_rows_as(Model,dataframe)
display(validated)
validated.dtypes

,number,string,date
0,1,a,1994-06-11
1,2,b,2023-02-10
2,3,3,2023-02-10


number     int64
string    object
date      object
dtype: object

> To Do: Find a way to return the index of the erros in the ValidationError

In [ ]:
bad_dataframe = pd.DataFrame({
    'number':['a',1,'b'],
    'string':dataframe['string'],
    'date':dataframe['date'],
},dtype='object')
try:
    df = parse_dataframe_rows_as(Model,bad_dataframe)
    display(df)
except ValidationError as e:
    print(e)

2 validation errors for Model
__root__ -> number
  value is not a valid integer (type=type_error.integer)
__root__ -> number
  value is not a valid integer (type=type_error.integer)


In [ ]:
#|exporti 

class TypedRecordFrame(DataFrame):
    row_model: Optional[Type[BaseModel]] = None
    alias_as_column_names: bool = False
        
    @classmethod
    def __get_validators__(cls):
        for v in super().__get_validators__():
            yield v
        yield cls.validate_rows
        yield cls.validate_column_names
    
    @classmethod
    def __modify_schema__(cls, field_schema: Dict[str, Any]) -> None:
        row_model=None
        if cls.row_model:
            row_model = cls.row_model.schema()
        update_not_none(
            field_schema,
            row_model=row_model
        )

    @classmethod
    def validate_rows(cls, df):
        
        if cls.row_model:
            validated = parse_dataframe_rows_as(cls.row_model,df)
                            
            return validated
        return df
    
    @classmethod
    def validate_column_names(cls,df):
        if cls.alias_as_column_names==True:
            # create a dictionary mapping field names to aliases
            field_name_to_alias = {
                field_name: cls.row_model.__fields__[field_name].alias
                for field_name in cls.row_model.__fields__
            }

            df.rename(columns=field_name_to_alias,inplace=True)
        return df
            

In [ ]:
#|exporti 

class RecordModelFrameMeta(type):
    def __getitem__(self, constraint):
        return type('RecordFrame', (TypedRecordFrame,), {'row_model': constraint})

In [ ]:
#|exporti
class RecordFrame(DataFrame, metaclass=RecordModelFrameMeta):
    pass

In [ ]:
#|export 

def record_model(kls):
    """Decorator to make a pydantic model into a RecordFrame, i.e. a DataFrame validated by row."""
    return RecordFrame[kls]

In [ ]:
class ModelWithConstrainedFrame(BaseModel):
    df: RecordFrame[Model]

In [ ]:
ModelWithConstrainedFrame(df=dataframe)

ModelWithConstrainedFrame(df=   number string        date
0       1      a  1994-06-11
1       2      b  2023-02-10
2       3      3  2023-02-10)

### Example: Recently Played Songs on WFUV

In [ ]:
from archetypon.base_model import DataFrameModel
from pydantic import Field
from pydantic import HttpUrl,Field
import pandas as pd

In [ ]:

@record_model
class Song(BaseModel):
    time: str = Field(alias='Time')
    song_title: str = Field(alias='Song Title')
    artist: Optional[str] = Field(alias='Artist')

In [ ]:
class WFUVRecentlyPlayed(DataFrameModel[Song]):
    """A DataFrame of recently played songs from WFUV"""
    
    source:ClassVar[HttpUrl] = "https://wfuv.org/playlist"
    timestamp: dt.datetime = Field(description='The time that the data was collected',default_factory = dt.datetime.now)
    
    def __init__(self,*args,**kwargs):
        data = pd.read_html(self.source)[0]
        super().__init__(data=data,*args,**kwargs)
        

In [ ]:
recently_played = WFUVRecentlyPlayed()
recently_played

,time,song_title,artist
0,"02/10, 1:38pm",The Greatest,Cat Power
1,"02/10, 1:31pm",Same Ol Mistakes,Rihanna
2,"02/10, 1:26pm",This Year,Emily King
3,"02/10, 1:23pm",Part of the Band,The 1975
4,"02/10, 1:19pm",Disarm,Smashing Pumpkins
...,...,...,...
345,"02/09, 1:16pm",Vacation,The Go-Go's
346,"02/09, 1:07pm",The Payback,James Brown
347,"02/09, 1:03pm",New Gold,Gorillaz
348,"02/09, 1:00pm",Modern Girl,Sleater-Kinney


In [ ]:
assert type(recently_played.data)==pd.DataFrame